## Objective

ASRC のシミュレーション用に，表面粗さをもつ水素終端ダイアモンド基板のデータを Lammps' DATA の形式で用意する．

### 上下面の分割

$z$ 座標が 60 angstrom より小さいかどうかで判断する．

In [1]:
# ファイルの読み込み
with open('data.SolidPlates','r') as f:
    data = f.read()
lines = data.split('\n')
# 上下面の Atom-ID を取得する
lower_atom_ids = []
upper_atom_ids = []
is_atoms_section = False
blank_counter = 0
for line in lines:
    
    if is_atoms_section:
        if line:
            vals = line.split()
            z = float(vals[6])
            if z < 60:
                lower_atom_ids.append(int(vals[0]))
            else:
                upper_atom_ids.append(int(vals[0]))
        else:
            blank_counter += 1
            if 1 < blank_counter:
                is_atoms_section = False
                blank_counter = 0
                
    if line == "Atoms":
        is_atoms_section = True
        
print(len(lower_atom_ids))
print(len(upper_atom_ids))

16800
16800


List から要素を検索する時間は $O(n)$ であり，非常に長い．そこで，List を Set に変換しておく．これにより，$O(1)$ に負荷が低減される．

In [2]:
lower_atom_ids = set(lower_atom_ids)
upper_atom_ids = set(upper_atom_ids)

Bond の両端の原子の ID で判断する．片側の原子のみで大丈夫なはずだが，チェック込みで両端を使う．

In [3]:
# 上下面の Bond-ID を取得する
lower_bond_ids = []
upper_bond_ids = []
is_bonds_section = False
blank_counter = 0
for line in lines:
    
    if is_bonds_section:
        if line:
            vals = line.split()
            atom_i_id = int(vals[2])
            atom_j_id = int(vals[3])
            if atom_i_id in lower_atom_ids and atom_j_id in lower_atom_ids:
                lower_bond_ids.append(int(vals[0]))
            elif atom_i_id in upper_atom_ids and atom_j_id in upper_atom_ids:
                upper_bond_ids.append(int(vals[0]))
            else:
                print("Warning")
        else:
            blank_counter += 1
            if 1 < blank_counter:
                is_bonds_section = False
                blank_counter = 0
                
    if line == "Bonds":
        is_bonds_section = True
        
print(len(lower_bond_ids))
print(len(upper_bond_ids))

31300
31300


真ん中の原子のみで判断する．

In [4]:
# 上下面の Angle-ID を取得する
lower_angle_ids = []
upper_angle_ids = []
is_angles_section = False
blank_counter = 0
for line in lines:
    
    if is_angles_section:
        if line:
            vals = line.split()
            atom_id = int(vals[3])
            if atom_id in lower_atom_ids:
                lower_angle_ids.append(int(vals[0]))
            elif atom_id in upper_atom_ids:
                upper_angle_ids.append(int(vals[0]))
            else:
                print("Warning")
        else:
            blank_counter += 1
            if 1 < blank_counter:
                is_angles_section = False
                blank_counter = 0
                
    if line == "Angles":
        is_angles_section = True
        
print(len(lower_angle_ids))
print(len(upper_angle_ids))

90800
90800


2 つ目の原子のみで判断する．

In [5]:
# 上下面の Dihedral-ID を取得する
lower_dihedral_ids = []
upper_dihedral_ids = []
is_dihedrals_section = False
blank_counter = 0
for line in lines:
    
    if is_dihedrals_section:
        if line:
            vals = line.split()
            atom_id = int(vals[3])
            if atom_id in lower_atom_ids:
                lower_dihedral_ids.append(int(vals[0]))
            elif atom_id in upper_atom_ids:
                upper_dihedral_ids.append(int(vals[0]))
            else:
                print("Warning")
        else:
            blank_counter += 1
            if 1 < blank_counter:
                is_dihedrals_section = False
                blank_counter = 0
                
    if line == "Dihedrals":
        is_dihedrals_section = True
        
print(len(lower_dihedral_ids))
print(len(upper_dihedral_ids))

263100
263100


2 つ目の原子のみで判断する．

In [6]:
# 上下面の Improper-ID を取得する
lower_improper_ids = []
upper_improper_ids = []
is_impropers_section = False
blank_counter = 0
for line in lines:
    
    if is_impropers_section:
        if line:
            vals = line.split()
            atom_id = int(vals[3])
            if atom_id in lower_atom_ids:
                lower_improper_ids.append(int(vals[0]))
            elif atom_id in upper_atom_ids:
                upper_improper_ids.append(int(vals[0]))
            else:
                print("Warning")
        else:
            blank_counter += 1
            if 1 < blank_counter:
                is_impropers_section = False
                blank_counter = 0
                
    if line == "Impropers":
        is_impropers_section = True
        
print(len(lower_improper_ids))
print(len(upper_improper_ids))

60000
60000


下側固体の DATA を作成する．

In [7]:
# データまとめ
lower_ids = {
    "Atoms": lower_atom_ids,
    "Bonds": set(lower_bond_ids),
    "Angles": set(lower_angle_ids),
    "Dihedrals": set(lower_dihedral_ids),
    "Impropers": set(lower_improper_ids)
}

In [8]:
# ファイルへ書き込み
with open('data.SolidPlates_lower','w') as f:
    
    is_header = True
    section_name = None
    section_counter = 0
    
    for line in lines:
        if is_header:
            if line == "33600 atoms":
                f.write("16800 atoms" + '\n')
            elif line == "62600 bonds":
                f.write("31300 bonds" + '\n')
            elif line == "181600 angles":
                f.write("90800 angles" + '\n')
            elif line == "526200 dihedrals":
                f.write("263100 dihedrals" + '\n')
            elif line == "120000 impropers":
                f.write("60000 impropers" + '\n')
            elif line == "Atoms":
                is_header = False
                section_name = line
                f.write(line + '\n')
            else:
                f.write(line + '\n')
        else:
            if line in ["Bonds", "Angles", "Dihedrals", "Impropers"]:
                print("{}: {}".format(section_name, section_counter))
                section_counter = 0
                section_name = line
                f.write(line + '\n')
            else:
                if line:
                    vals = line.split()
                    ID = int(vals[0])
                    if ID in lower_ids[section_name]:
                        f.write(line + '\n')
                        section_counter += 1
                else:
                    f.write(line + '\n')
                    
print("{}: {}".format(section_name, section_counter))
            

Atoms: 16800
Bonds: 31300
Angles: 90800
Dihedrals: 263100
Impropers: 60000


上側固体の DATA を作成する．

In [9]:
# データまとめ
upper_ids = {
    "Atoms": upper_atom_ids,
    "Bonds": set(upper_bond_ids),
    "Angles": set(upper_angle_ids),
    "Dihedrals": set(upper_dihedral_ids),
    "Impropers": set(upper_improper_ids)
}

In [10]:
# ファイルへ書き込み
with open('data.SolidPlates_upper','w') as f:
    
    is_header = True
    section_name = None
    section_counter = 0
    
    for line in lines:
        if is_header:
            if line == "33600 atoms":
                f.write("16800 atoms" + '\n')
            elif line == "62600 bonds":
                f.write("31300 bonds" + '\n')
            elif line == "181600 angles":
                f.write("90800 angles" + '\n')
            elif line == "526200 dihedrals":
                f.write("263100 dihedrals" + '\n')
            elif line == "120000 impropers":
                f.write("60000 impropers" + '\n')
            elif line == "Atoms":
                is_header = False
                section_name = line
                f.write(line + '\n')
            else:
                f.write(line + '\n')
        else:
            if line in ["Bonds", "Angles", "Dihedrals", "Impropers"]:
                print("{}: {}".format(section_name, section_counter))
                section_counter = 0
                section_name = line
                f.write(line + '\n')
            else:
                if line:
                    vals = line.split()
                    ID = int(vals[0])
                    if ID in upper_ids[section_name]:
                        f.write(line + '\n')
                        section_counter += 1
                else:
                    f.write(line + '\n')
                    
print("{}: {}".format(section_name, section_counter))
            

Atoms: 16800
Bonds: 31300
Angles: 90800
Dihedrals: 263100
Impropers: 60000


### Unit Cell (Disk)

$0 < x < 10.0578 \ \unicode[.8,0]{x212B}, \ 0 < y < 10.0578 \ \unicode[.8,0]{x212B}$ の領域を Unit Cell としてデータを取り出す．
Bond, Angle, Dihedral, Improper が Unit Cell に含まれるかについては，原子シーケンスの 1 番目の原子が含まれるかどうかで判断する．
Bond, Angle, Dihedral, Improper の原子シーケンスのデータは Dictionary 型で，

```
{"cell": (ix, iy), "serial": i}
```

の形で得る．ここで，`(ix, iy)` はオリジナルのセルから見たときのセルの位置座標で，-1, 0, 1 のいずれかになるはずである．`serial` は Unit Cell 内での ID のことである．


#### Atom の取得

つぎのデータ形式のリストとして得る．分子と周期境界条件に関するデータはすべて 0 なので，記憶しておく必要はない．

```
atom_list = {
    "serial": int,
    "id": int,
    "type": int,
    "charge": float,
    "x": float,
    "y": float,
    "z": float
}
```

In [158]:
# ファイルの読み込み
with open('data.SolidPlates_lower','r') as f:
    data = f.read()
lines = data.split('\n')
# データの取得
atom_list = []
serial = 0
is_atoms_section = False
blank_counter = 0
for line in lines:
    
    if is_atoms_section:
        if line:
            vals = line.split()
            x = float(vals[4])
            y = float(vals[5])
            if (0.0 < x and x < 10.0578) and (0.0 < y and y < 10.0578):
                serial += 1
                atom_list.append({
                        "serial": serial,
                        "id": int(vals[0]),
                        "type": int(vals[2]),
                        "charge": float(vals[3]),
                        "x": x,
                        "y": y,
                        "z": float(vals[6])
                })
        else:
            blank_counter += 1
            if 1 < blank_counter:
                is_atoms_section = False
                blank_counter = 0
                
    if line == "Atoms":
        is_atoms_section = True


Unit Cell に属する原子の Original ID を集めた Set を作成しておく．

In [159]:
ids_in_unit_cell = []
for atom in atom_list:
    ids_in_unit_cell.append(atom['id'])
ids_in_unit_cell = set(ids_in_unit_cell)
print(len(ids_in_unit_cell))

336


#### 原子シーケンス用のデータ

```
{"cell": (ix, iy), "serial": i}
```

すべての原子について，ID から原子シーケンス用のデータへの Dictionary を作成しておく．

In [160]:
tolerance = 0.1
# ファイルの読み込み
with open('data.SolidPlates_lower','r') as f:
    data = f.read()
lines = data.split('\n')
# データの取得
id_to_sequence = {}
is_atoms_section = False
blank_counter = 0
for line in lines:
    
    if is_atoms_section:
        if line:
            vals = line.split()
            x = float(vals[4])
            y = float(vals[5])
            z = float(vals[6])
            ix = int(x/10.0578)
            iy = int(y/10.0578)
            wrapped_x = x - ix*10.0578
            wrapped_y = y - iy*10.0578
            correspond_counter = 0
            for atom in atom_list:
                if (tolerance < abs(atom["x"] - wrapped_x)):
                    continue
                if (tolerance < abs(atom["y"] - wrapped_y)):
                    continue
                if (tolerance < abs(atom["z"] - z)):
                    continue
                correspond_counter += 1
                id_to_sequence[vals[0]] = {
                    "cell": (ix if ix < 9 else -1, iy if iy < 4 else -1),
                    "serial": atom["serial"]
                }
            if not correspond_counter == 1:
                print("Warning")
                
        else:
            blank_counter += 1
            if 1 < blank_counter:
                is_atoms_section = False
                blank_counter = 0
                
    if line == "Atoms":
        is_atoms_section = True
        

#### Bond の取得

```
bond_list = {
    "serial": int,
    "id": int,
    "type": int,
    "atom1": {(0, 0) & serial},
    "atom2": {cell & serial}
}
```

In [161]:
# ファイルの読み込み
with open('data.SolidPlates_lower','r') as f:
    data = f.read()
lines = data.split('\n')
# データの取得
bond_list = []
serial = 0
is_bonds_section = False
blank_counter = 0
for line in lines:
    
    if is_bonds_section:
        if line:
            vals = line.split()
            if int(vals[2]) in ids_in_unit_cell:
                serial += 1
                bond_list.append({
                    "serial": serial,
                    "id": int(vals[0]),
                    "type": int(vals[1]),
                    "atom1": id_to_sequence[vals[2]],
                    "atom2": id_to_sequence[vals[3]]
                })
                if not abs(id_to_sequence[vals[3]]["cell"][0]) < 2:
                    print("Warning")
                if not abs(id_to_sequence[vals[3]]["cell"][1]) < 2:
                    print("Warning")
        else:
            blank_counter += 1
            if 1 < blank_counter:
                is_bonds_section = False
                blank_counter = 0
                
    if line == "Bonds":
        is_bonds_section = True


#### Angle の取得

```
angle_list = {
    "serial": int,
    "id": int,
    "type": int,
    "atom1": {(0, 0) & serial},
    "atom2": {cell & serial},
    "atom3": {cell & serial}
}
```

In [162]:
# ファイルの読み込み
with open('data.SolidPlates_lower','r') as f:
    data = f.read()
lines = data.split('\n')
# データの取得
angle_list = []
serial = 0
is_angles_section = False
blank_counter = 0
for line in lines:
    
    if is_angles_section:
        if line:
            vals = line.split()
            if int(vals[2]) in ids_in_unit_cell:
                serial += 1
                angle_list.append({
                    "serial": serial,
                    "id": int(vals[0]),
                    "type": int(vals[1]),
                    "atom1": id_to_sequence[vals[2]],
                    "atom2": id_to_sequence[vals[3]],
                    "atom3": id_to_sequence[vals[4]]
                })
                for i in [3, 4]:
                    if not abs(id_to_sequence[vals[i]]["cell"][0]) < 2:
                        print("Warning")
                    if not abs(id_to_sequence[vals[i]]["cell"][1]) < 2:
                        print("Warning")
        else:
            blank_counter += 1
            if 1 < blank_counter:
                is_angles_section = False
                blank_counter = 0
                
    if line == "Angles":
        is_angles_section = True
        

#### Dihedral の取得

```
dihedral_list = {
    "serial": int,
    "id": int,
    "type": int,
    "atom1": {(0, 0) & serial},
    "atom2": {cell & serial},
    "atom3": {cell & serial},
    "atom4": {cell & serial}
}
```

In [163]:
# ファイルの読み込み
with open('data.SolidPlates_lower','r') as f:
    data = f.read()
lines = data.split('\n')
# データの取得
dihedral_list = []
serial = 0
is_dihedrals_section = False
blank_counter = 0
for line in lines:
    
    if is_dihedrals_section:
        if line:
            vals = line.split()
            if int(vals[2]) in ids_in_unit_cell:
                serial += 1
                dihedral_list.append({
                    "serial": serial,
                    "id": int(vals[0]),
                    "type": int(vals[1]),
                    "atom1": id_to_sequence[vals[2]],
                    "atom2": id_to_sequence[vals[3]],
                    "atom3": id_to_sequence[vals[4]],
                    "atom4": id_to_sequence[vals[5]]
                })
                for i in [3, 4, 5]:
                    if not abs(id_to_sequence[vals[i]]["cell"][0]) < 2:
                        print("Warning")
                    if not abs(id_to_sequence[vals[i]]["cell"][1]) < 2:
                        print("Warning")
        else:
            blank_counter += 1
            if 1 < blank_counter:
                is_dihedrals_section = False
                blank_counter = 0
                
    if line == "Dihedrals":
        is_dihedrals_section = True
        

#### Improper の取得

```
improper = {
    "serial": int,
    "id": int,
    "type": int,
    "atom1": {(0, 0) & serial},
    "atom2": {cell & serial},
    "atom3": {cell & serial},
    "atom4": {cell & serial}
}
```

In [164]:
# ファイルの読み込み
with open('data.SolidPlates_lower','r') as f:
    data = f.read()
lines = data.split('\n')
# データの取得
improper_list = []
serial = 0
is_impropers_section = False
blank_counter = 0
for line in lines:
    
    if is_impropers_section:
        if line:
            vals = line.split()
            if int(vals[2]) in ids_in_unit_cell:
                serial += 1
                improper_list.append({
                    "serial": serial,
                    "id": int(vals[0]),
                    "type": int(vals[1]),
                    "atom1": id_to_sequence[vals[2]],
                    "atom2": id_to_sequence[vals[3]],
                    "atom3": id_to_sequence[vals[4]],
                    "atom4": id_to_sequence[vals[5]]
                })
                for i in [3, 4, 5]:
                    if not abs(id_to_sequence[vals[i]]["cell"][0]) < 2:
                        print("Warning")
                    if not abs(id_to_sequence[vals[i]]["cell"][1]) < 2:
                        print("Warning")
        else:
            blank_counter += 1
            if 1 < blank_counter:
                is_impropers_section = False
                blank_counter = 0
                
    if line == "Impropers":
        is_impropers_section = True
        

### ファイル作成 (Disk)

In [142]:
import math

In [143]:
# Cell & Serial => ID
def getID(num, ix, iy, serial):
    return ix*8*num+ iy*num + serial

#### Header

In [111]:
header = """# LAMMPS InputData

{num_atoms} atoms
{num_bonds} bonds
{num_angles} angles
{num_dihedrals} dihedrals
{num_impropers} impropers

16 atom types
6 bond types
13 angle types
18 dihedral types
13 improper types

      0.00000000      80.46240000  xlo xhi
      0.00000000      80.46240000  ylo yhi
      0.00000000     150.00000000  zlo zhi

""".format(
    num_atoms=len(atom_list)*8*8,
    num_bonds=len(bond_list)*8*8,
    num_angles=len(angle_list)*8*8,
    num_dihedrals=len(dihedral_list)*8*8,
    num_impropers=len(improper_list)*8*8,
)

In [112]:
# Lower solid
# ファイルの読み込み
with open('data.Disk', 'w') as f:
    f.write(header)

#### Atoms

In [113]:
num_atoms = len(atom_list)
with open('data.Disk', 'a') as f:
    f.write("Atoms\n")
    f.write("\n")
    for i in range(8):
        for j in range(8):
            for atom in atom_list:
                f.write(
                    "{id:6d} 0 {type:2d} {charge:9.6f} {x:12.8f} {y:12.8f} {z:12.8f} 0 0 0\n".format(
                        id=getID(num_atoms, i, j, atom["serial"]),
                        type=atom["type"],
                        charge=atom["charge"],
                        x=i*10.0578 + atom["x"],
                        y=j*10.0578 + atom["y"],
                        z=atom["z"]
                    ))
    f.write("\n")

#### Bonds

In [114]:
num_atoms = len(atom_list)
num_bonds = len(bond_list)
with open('data.Disk', 'a') as f:
    f.write("Bonds\n")
    f.write("\n")
    for i in range(8):
        for j in range(8):
            for bond in bond_list:
                i_atom1 = i + bond["atom1"]["cell"][0]
                j_atom1 = j + bond["atom1"]["cell"][1]
                i_atom2 = i + bond["atom2"]["cell"][0]
                j_atom2 = j + bond["atom2"]["cell"][1]
                f.write(
                    "{id:6d} {type:2d} {id1:6d} {id2:6d}\n".format(
                        id=getID(num_bonds, i, j, bond["serial"]),
                        type=bond["type"],
                        id1=getID(
                            num_atoms,
                            i_atom1-math.floor(i_atom1/8)*8,
                            j_atom1-math.floor(j_atom1/8)*8,
                            bond["atom1"]["serial"]
                        ),
                        id2=getID(
                            num_atoms,
                            i_atom2-math.floor(i_atom2/8)*8,
                            j_atom2-math.floor(j_atom2/8)*8,
                            bond["atom2"]["serial"]
                        )
                    ))
    f.write("\n")

#### Angles

In [115]:
num_atoms = len(atom_list)
num_angles = len(angle_list)
with open('data.Disk', 'a') as f:
    f.write("Angles\n")
    f.write("\n")
    for i in range(8):
        for j in range(8):
            for angle in angle_list:
                i_atom1 = i + angle["atom1"]["cell"][0]
                j_atom1 = j + angle["atom1"]["cell"][1]
                i_atom2 = i + angle["atom2"]["cell"][0]
                j_atom2 = j + angle["atom2"]["cell"][1]
                i_atom3 = i + angle["atom3"]["cell"][0]
                j_atom3 = j + angle["atom3"]["cell"][1]
                f.write(
                    "{id:6d} {type:2d} {id1:6d} {id2:6d} {id3:6d}\n".format(
                        id=getID(num_angles, i, j, angle["serial"]),
                        type=angle["type"],
                        id1=getID(
                            num_atoms,
                            i_atom1-math.floor(i_atom1/8)*8,
                            j_atom1-math.floor(j_atom1/8)*8,
                            angle["atom1"]["serial"]
                        ),
                        id2=getID(
                            num_atoms,
                            i_atom2-math.floor(i_atom2/8)*8,
                            j_atom2-math.floor(j_atom2/8)*8,
                            angle["atom2"]["serial"]
                        ),
                        id3=getID(
                            num_atoms,
                            i_atom3-math.floor(i_atom3/8)*8,
                            j_atom3-math.floor(j_atom3/8)*8,
                            angle["atom3"]["serial"]
                        )
                    ))
    f.write("\n")

#### Dihedrals

In [116]:
num_atoms = len(atom_list)
num_dihedrals = len(dihedral_list)
with open('data.Disk', 'a') as f:
    f.write("Dihedrals\n")
    f.write("\n")
    for i in range(8):
        for j in range(8):
            for dihedral in dihedral_list:
                i_atom1 = i + dihedral["atom1"]["cell"][0]
                j_atom1 = j + dihedral["atom1"]["cell"][1]
                i_atom2 = i + dihedral["atom2"]["cell"][0]
                j_atom2 = j + dihedral["atom2"]["cell"][1]
                i_atom3 = i + dihedral["atom3"]["cell"][0]
                j_atom3 = j + dihedral["atom3"]["cell"][1]
                i_atom4 = i + dihedral["atom4"]["cell"][0]
                j_atom4 = j + dihedral["atom4"]["cell"][1]
                f.write(
                    "{id:6d} {type:2d} {id1:6d} {id2:6d} {id3:6d} {id4:6d}\n".format(
                        id=getID(num_dihedrals, i, j, dihedral["serial"]),
                        type=dihedral["type"],
                        id1=getID(
                            num_atoms,
                            i_atom1-math.floor(i_atom1/8)*8,
                            j_atom1-math.floor(j_atom1/8)*8,
                            dihedral["atom1"]["serial"]
                        ),
                        id2=getID(
                            num_atoms,
                            i_atom2-math.floor(i_atom2/8)*8,
                            j_atom2-math.floor(j_atom2/8)*8,
                            dihedral["atom2"]["serial"]
                        ),
                        id3=getID(
                            num_atoms,
                            i_atom3-math.floor(i_atom3/8)*8,
                            j_atom3-math.floor(j_atom3/8)*8,
                            dihedral["atom3"]["serial"]
                        ),
                        id4=getID(
                            num_atoms,
                            i_atom4-math.floor(i_atom4/8)*8,
                            j_atom4-math.floor(j_atom4/8)*8,
                            dihedral["atom4"]["serial"]
                        )
                    ))
    f.write("\n")

#### Impropers

In [117]:
num_atoms = len(atom_list)
num_impropers = len(improper_list)
with open('data.Disk', 'a') as f:
    f.write("Impropers\n")
    f.write("\n")
    for i in range(8):
        for j in range(8):
            for improper in improper_list:
                i_atom1 = i + improper["atom1"]["cell"][0]
                j_atom1 = j + improper["atom1"]["cell"][1]
                i_atom2 = i + improper["atom2"]["cell"][0]
                j_atom2 = j + improper["atom2"]["cell"][1]
                i_atom3 = i + improper["atom3"]["cell"][0]
                j_atom3 = j + improper["atom3"]["cell"][1]
                i_atom4 = i + improper["atom4"]["cell"][0]
                j_atom4 = j + improper["atom4"]["cell"][1]
                f.write(
                    "{id:6d} {type:2d} {id1:6d} {id2:6d} {id3:6d} {id4:6d}\n".format(
                        id=getID(num_impropers, i, j, improper["serial"]),
                        type=improper["type"],
                        id1=getID(
                            num_atoms,
                            i_atom1-math.floor(i_atom1/8)*8,
                            j_atom1-math.floor(j_atom1/8)*8,
                            improper["atom1"]["serial"]
                        ),
                        id2=getID(
                            num_atoms,
                            i_atom2-math.floor(i_atom2/8)*8,
                            j_atom2-math.floor(j_atom2/8)*8,
                            improper["atom2"]["serial"]
                        ),
                        id3=getID(
                            num_atoms,
                            i_atom3-math.floor(i_atom3/8)*8,
                            j_atom3-math.floor(j_atom3/8)*8,
                            improper["atom3"]["serial"]
                        ),
                        id4=getID(
                            num_atoms,
                            i_atom4-math.floor(i_atom4/8)*8,
                            j_atom4-math.floor(j_atom4/8)*8,
                            improper["atom4"]["serial"]
                        )
                    ))
    f.write("\n")

### Unit Cell (Head)

#### Atom の取得

つぎのデータ形式のリストとして得る．分子と周期境界条件に関するデータはすべて 0 なので，記憶しておく必要はない．

```
atom_list = {
    "serial": int,
    "id": int,
    "type": int,
    "charge": float,
    "x": float,
    "y": float,
    "z": float
}
```

In [165]:
# ファイルの読み込み
with open('data.SolidPlates_upper','r') as f:
    data = f.read()
lines = data.split('\n')
# データの取得
atom_list = []
serial = 0
is_atoms_section = False
blank_counter = 0
for line in lines:
    
    if is_atoms_section:
        if line:
            vals = line.split()
            x = float(vals[4])
            y = float(vals[5])
            if (0.0 < x and x < 10.0578) and (0.0 < y and y < 10.0578):
                serial += 1
                atom_list.append({
                        "serial": serial,
                        "id": int(vals[0]),
                        "type": int(vals[2]),
                        "charge": float(vals[3]),
                        "x": x,
                        "y": y,
                        "z": float(vals[6])
                })
        else:
            blank_counter += 1
            if 1 < blank_counter:
                is_atoms_section = False
                blank_counter = 0
                
    if line == "Atoms":
        is_atoms_section = True


Unit Cell に属する原子の Original ID を集めた Set を作成しておく．

In [166]:
ids_in_unit_cell = []
for atom in atom_list:
    ids_in_unit_cell.append(atom['id'])
ids_in_unit_cell = set(ids_in_unit_cell)
print(len(ids_in_unit_cell))

336


#### 原子シーケンス用のデータ

```
{"cell": (ix, iy), "serial": i}
```

In [167]:
tolerance = 0.1
# ファイルの読み込み
with open('data.SolidPlates_upper','r') as f:
    data = f.read()
lines = data.split('\n')
# データの取得
id_to_sequence = {}
is_atoms_section = False
blank_counter = 0
for line in lines:
    
    if is_atoms_section:
        if line:
            vals = line.split()
            x = float(vals[4])
            y = float(vals[5])
            z = float(vals[6])
            ix = int(x/10.0578)
            iy = int(y/10.0578)
            wrapped_x = x - ix*10.0578
            wrapped_y = y - iy*10.0578
            correspond_counter = 0
            for atom in atom_list:
                if (tolerance < abs(atom["x"] - wrapped_x)):
                    continue
                if (tolerance < abs(atom["y"] - wrapped_y)):
                    continue
                if (tolerance < abs(atom["z"] - z)):
                    continue
                correspond_counter += 1
                id_to_sequence[vals[0]] = {
                    "cell": (ix if ix < 9 else -1, iy if iy < 4 else -1),
                    "serial": atom["serial"]
                }
            if not correspond_counter == 1:
                print("Warning")
                
        else:
            blank_counter += 1
            if 1 < blank_counter:
                is_atoms_section = False
                blank_counter = 0
                
    if line == "Atoms":
        is_atoms_section = True
        

#### BADI (Bond, Angle, Dihedral, Improper) の取得

```
badi = {
    "Bonds": [{
        "serial": int,
        "id": int,
        "type": int,
        "atoms": [{(0, 0) & serial}, {cell & serial}]
    }, ...],
    "Angles": [{
        "serial": int,
        "id": int,
        "type": int,
        "atoms": [{(0, 0) & serial}, {cell & serial}, {cell & serial}]
    }, ...],
    "Dihedrals": [{
        "serial": int,
        "id": int,
        "type": int,
        "atoms": [{(0, 0) & serial}, {cell & serial}, {cell & serial}, {cell & serial}]
    }, ...],
    "Impropers": [{
        "serial": int,
        "id": int,
        "type": int,
        "atoms": [{(0, 0) & serial}, {cell & serial}, {cell & serial}, {cell & serial}]
    }, ...]
}
```

In [138]:
# ファイルの読み込み
with open('data.SolidPlates_upper','r') as f:
    data = f.read()
lines = data.split('\n')
# データの取得
badi = {
    "Bonds": [],
    "Angles": [],
    "Dihedrals": [],
    "Impropers": []
}
serial = {
    "Bonds": 0,
    "Angles": 0,
    "Dihedrals": 0,
    "Impropers": 0
}
section_name = None
warning_counter = 0
for line in lines:
    
    if line in ["Bonds", "Angles", "Dihedrals", "Impropers"]:
        section_name = line
    elif section_name:
        if line:
            vals = line.split()
            if int(vals[2]) in ids_in_unit_cell:
                serial[section_name] += 1
                badi[section_name].append({
                    "serial": serial[section_name],
                    "id": int(vals[0]),
                    "type": int(vals[1]),
                    "atoms": [id_to_sequence[val] for val in vals[2:]]
                })
                for val in vals[2:]:
                    if not abs(id_to_sequence[val]["cell"][0]) < 2:
                        warning_counter += 1
                    if not abs(id_to_sequence[val]["cell"][1]) < 2:
                        warning_counter += 1
        
print(warning_counter)

0


In [140]:
for name in ["Bonds", "Angles", "Dihedrals", "Impropers"]:
    print(len(badi[name]))

626
1816
5262
1200


### ファイル作成 (Head)

#### Header

In [144]:
header = """# LAMMPS InputData

{num_atoms} atoms
{num_bonds} bonds
{num_angles} angles
{num_dihedrals} dihedrals
{num_impropers} impropers

16 atom types
6 bond types
13 angle types
18 dihedral types
13 improper types

      0.00000000      80.46240000  xlo xhi
      0.00000000      80.46240000  ylo yhi
      0.00000000     150.00000000  zlo zhi

""".format(
    num_atoms=len(atom_list)*8*8,
    num_bonds=len(badi["Bonds"])*8*8,
    num_angles=len(badi["Angles"])*8*8,
    num_dihedrals=len(badi["Dihedrals"])*8*8,
    num_impropers=len(badi["Impropers"])*8*8,
)

In [145]:
# Lower solid
# ファイルの読み込み
with open('data.Head', 'w') as f:
    f.write(header)

#### Atoms

In [146]:
num_atoms = len(atom_list)
with open('data.Head', 'a') as f:
    f.write("Atoms\n")
    f.write("\n")
    for i in range(8):
        for j in range(8):
            for atom in atom_list:
                f.write(
                    "{id:6d} 0 {type:2d} {charge:9.6f} {x:12.8f} {y:12.8f} {z:12.8f} 0 0 0\n".format(
                        id=getID(num_atoms, i, j, atom["serial"]),
                        type=atom["type"],
                        charge=atom["charge"],
                        x=i*10.0578 + atom["x"],
                        y=j*10.0578 + atom["y"],
                        z=atom["z"]
                    ))
    f.write("\n")

#### BADI (Bond, Angle, Dihedral, Improper)

In [148]:
num_atoms = len(atom_list)
nums = {name: len(badi[name]) for name in ["Bonds", "Angles", "Dihedrals", "Impropers"]}
with open('data.Head', 'a') as f:
    for name in ["Bonds", "Angles", "Dihedrals", "Impropers"]:
        f.write(name + "\n")
        f.write("\n")
        for i in range(8):
            for j in range(8):
                for item in badi[name]:
                    i_atoms = [i + atom["cell"][0] for atom in item["atoms"]]
                    j_atoms = [j + atom["cell"][1] for atom in item["atoms"]]
                    new_line = "{id:6d} {type:2d}".format(
                        id=getID(nums[name], i, j, item["serial"]),
                        type=item["type"]
                    )
                    for i_atom, j_atom, atom in zip(i_atoms, j_atoms, item["atoms"]):
                        new_line += " {:6d}".format(
                            getID(
                                num_atoms,
                                i_atom-math.floor(i_atom/8)*8, # 周期境界条件の解決
                                j_atom-math.floor(j_atom/8)*8, # 周期境界条件の解決
                                atom["serial"]
                            )
                        )
                    f.write(new_line + "\n")
        f.write("\n")

### 注意点

原子シーケンスの順不同性が問題になる可能性がある．例えば，両端の原子が 

```
{"cell": (0, 0), "serial": 3}, {"cell": (0, 1), "serial": 4}
```

と

```
{"cell": (0, 0), "serial": 4}, {"cell": (0, -1), "serial": 3}
```

の 2 つの結合はどちらも Unit Cell に帰属することになってしまうが，同一の結合なので，重複していることになってしまう．

とりあえず，Unit Cell 内に含まれる Bonds, Angle, Dihedral, Improper の数は正しいため，問題ないと思われるが，シミュレーション中に固体基板の不安定化などがおきた場合は，この点を確認する必要がある．


### 力場ファイル


* DeH: もともとの Demnum 用力場 = forcefield.DeH
* Z_R: 粗さのある表面と Z の力場 = forcefield.Z_roughDiaH
* New: 新しい力場

#### Atom Type

|  Name  |            Description            | DeH | Z_R | New |
|:------:|:---------------------------------:|:---:|:---:|:---:|
|  c44   |   C connected to 4 heavy atoms    |  1  |  1  |  1  |
|  c43   |   C connected to 3 heavy atoms    |  2  |  2  |  2  |
|   h1   |                 H                 |  3  |  3  |  3  |
|  f14   |            F in Demnum            |  4  |  4  |  4  |
|  o2e   |            O in Demnum            |  5  |  5  |  5  |
| c44_lt |  C in lower thermostated region   |  6  |  6  |  6  |
| c44_lb |      C in lower base region       |  7  |  7  |  7  |
| c44_ut |  C in upper thermostated region   |  8  |  8  |  8  |
| c44_ub |      C in upper base region       |  9  |  9  |  9  |
| c44_d  |        C in Demnum (or Z)         | 10  | 10  | 10  |
| c43_lh | C in lower solid & connected to H | 11  | 11  | 11  |
| c43_uh | C in upper solid & connected to H | 12  | 12  | 12  |
|  h1_l  |         H in lower solid          | 13  | 13  | 13  |
|  h1_u  |         H in upper solid          | 14  | 14  | 14  |
| c44_li |   C in lower interfacial region   |  -  | 15  | 15  |
| c44_ui |   C in upper interfacial region   |  -  | 16  | 16  |

mass と pair については forcefield.Z_roughDiaH が流用できる．

#### Bond Type

| Sequence | DeH | Z_R | New |
|:--------:|:---:|:---:|:---:|
| c44-c44  |  1  |  1  |     |
| c44-c43  |  2  |  2  |     |
| c43-c43  |  3  |  3  |     |
|  c43-h1  |  4  |  4  |     |
| c44-o2e  |  6  |  5  |     |
| c44-f14  |  5  |  6  |     |

forcefield.DeH が流用できる．Type が入れ替わっている部分は分子内であり，固体には影響がない．

#### Angle Type

|  Sequence   | DeH | Z_R | New |
|:-----------:|:---:|:---:|:---:|
| f14-c44-o2e |  5  |  1  |     |
| f14-c44-f14 |  2  |  2  |     |
| c44-c44-f14 |  1  |  3  |     |
| c44-c44-o2e |  4  |  4  |     |
| c44-o2e-c44 |  6  |  5  |     |
| o2e-c44-o2e |  -  |  6  |     |
| c44-c44-c44 |  3  |  7  |     |
| c43-c44-c43 |  7  |  8  |     |
| c43-c44-c44 |  8  |  9  |     |
| c43-c43-c44 |  9  | 10  |     |
| c44-c43-h1  | 10  | 11  |     |
| c43-c43-h1  | 11  | 12  |     |
| c44-c43-c44 | 12  | 13  |     |

forcefield.DeH を使う．固体の DATA ファイルにおいて Angle-type の変更が必要になる．

```python
convert_dict = {
  "7": 3,
  "8": 7,
  "9": 8,
  "10": 9,
  "11": 10,
  "12": 11,
  "13": 12
}
```

#### Dihedral Type

|    Sequence     | DeH | Z_R | New |
|:---------------:|:---:|:---:|:---:|
| c44-c44-c44-f14 |  1  |  -  |     |
| c44-c44-c44-o2e |  3  |  -  |     |
| f14-c44-c44-o2e |  4  |  1  |     |
| f14-c44-c44-f14 |  2  |  2  |     |
| c44-o2e-c44-f14 |  6  |  3  |     |
| o2e-c44-c44-o2e |  -  |  4  |     |
| c44-c44-o2e-c44 |  5  |  5  |     |
| c44-o2e-c44-o2e |  -  |  6  |     |
| c44-c44-c44-c44 |  7  |  7  |     |
| c43-c43-c44-c43 |  8  |  8  |     |
| c43-c44-c43-h1  |  9  |  9  |     |
| c43-c43-c44-c44 | 10  | 10  |     |
| c44-c44-c43-h1  | 11  | 11  |     |
| c43-c44-c43-c44 | 12  | 12  |     |
| c44-c43-c44-c44 | 13  | 13  |     |
| c43-c44-c44-c44 | 14  | 14  |     |
| c44-c43-c43-c44 | 15  | 15  |     |
| c44-c43-c43-h1  | 16  | 16  |     |
|  h1-c43-c43-h1  | 17  | 17  |     |
| c43-c44-c44-c43 |  -  | 18  |     |

forcefield.DeH に forcefield.Z_roughDiaH の 18 番目のものを追加して使える．

#### Improper Type

|    Sequence     | DeH | Z_R | New |
|:---------------:|:---:|:---:|:---:|
| c44-c44-c44-c44 |  1  |  1  |     |
| c43-c44-c43-c44 |  2  |  2  |     |
| c43-c44-c44-c44 |  3  |  3  |     |
| c43-c43-c44-h1  |  4  |  4  |     |
| c43-c43-c44-c44 |  5  |  5  |     |
| c44-c43-c44-h1  |  6  |  6  |     |
| c44-c44-c44-f14 | 10  |  -  |     |
| f14-c44-f14-o2e |  9  |  7  |     |
| f14-c44-f14-f14 | 11  |  8  |     |
| c44-c44-f14-f14 |  7  |  9  |     |
| c44-c44-f14-o2e |  8  | 10  |     |
| f14-c44-o2e-o2e |  -  | 11  |     |
| c44-c43-c44-c44 |  -  | 12  |     |
| c43-c44-c43-c43 |  -  | 13  |     |

forcefield.DeH に forcefield.Z_roughDiaH の 12, 13 番目のものを追加して使える．


#### Angle-type の修正

In [150]:
convert_dict = {
  "7": 3,
  "8": 7,
  "9": 8,
  "10": 9,
  "11": 10,
  "12": 11,
  "13": 12
}

In [154]:
with open("data.Disk", "r") as f:
    data = f.read()
lines = data.split("\n")
# 修正
in_Angles = False
blank_counter = 0
with open("data.Disk_new", "w") as f:
    for line in lines:
        
        if in_Angles:
            if line:
                vals = line.split()
                vals[1] = convert_dict[vals[1]]
                f.write("{:6d} {:2d} {:6d} {:6d} {:6d}\n".format(
                    int(vals[0]), vals[1], int(vals[2]), int(vals[3]), int(vals[4])
                ))
            else:
                f.write(line + "\n")
                blank_counter += 1
                if 1 < blank_counter:
                    in_Angles = False
        else:
            f.write(line + "\n")
        
        if line == "Angles":
            in_Angles = True
            

In [155]:
with open("data.Head", "r") as f:
    data = f.read()
lines = data.split("\n")
# 修正
in_Angles = False
blank_counter = 0
with open("data.Head_new", "w") as f:
    for line in lines:
        
        if in_Angles:
            if line:
                vals = line.split()
                vals[1] = convert_dict[vals[1]]
                f.write("{:6d} {:2d} {:6d} {:6d} {:6d}\n".format(
                    int(vals[0]), vals[1], int(vals[2]), int(vals[3]), int(vals[4])
                ))
            else:
                f.write(line + "\n")
                blank_counter += 1
                if 1 < blank_counter:
                    in_Angles = False
        else:
            f.write(line + "\n")
        
        if line == "Angles":
            in_Angles = True